In [5]:
import sqlite3
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

import pandas as pd
df_github = pd.read_csv('../../data/GH_data_safetensor.csv')
df_so = pd.read_csv('../../data/SO_data_safetensor.csv')
df_github.head()
df = df_github


,source,title,url,json content
0,GitHub,Safetensors,https://github.com/pytorch/torchchat/pull/1255,"{""title"": ""Safetensors"", ""url"": ""https://githu..."
1,GitHub,Safetensors support,https://github.com/urchade/GLiNER/pull/190,"{""title"": ""Safetensors support"", ""url"": ""https..."
2,GitHub,Add support for safetensors,https://github.com/MinishLab/model2vec/pull/36,"{""title"": ""Add support for safetensors"", ""url""..."
3,GitHub,update readme - safetensors,https://github.com/facebookresearch/vfusion3d/...,"{""title"": ""update readme - safetensors"", ""url""..."
4,GitHub,"Clarify ""Load Model"" SafeTensors documentation",https://github.com/chaiNNer-org/chaiNNer/pull/...,"{""title"": ""Clarify \""Load Model\"" SafeTensors ..."
...,...,...,...,...
4169,GitHub,Added flip function (to a tensor up to 5D alon...,https://github.com/torch/image/pull/60,"{""title"": ""Added flip function (to a tensor up..."
4170,GitHub,Add torch.view and torch.viewAs,https://github.com/torch/torch7/pull/50,"{""title"": ""Add torch.view and torch.viewAs"", ""..."
4171,GitHub,Stash computed form on tensor for re-assembly,https://github.com/firedrakeproject/firedrake/...,"{""title"": ""Stash computed form on tensor for r..."
4172,GitHub,Clover QPX,https://github.com/etmc/tmLQCD/pull/228,"{""title"": ""Clover QPX"", ""url"": ""https://github..."


In [9]:


print(df.shape)

# Combine relevant text columns for vectorization
df['combined_text'] = df[['json content']].fillna('').apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

# Define query related to model serialization formats
query = "model serialization safetensors"

# Vectorize the text data
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df['combined_text'])
query_vec = vectorizer.transform([query])

# Calculate cosine similarity between the query and all documents
cosine_similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()

# Get top N relevant papers
N = 500
top_n_indices = cosine_similarities.argsort()[-N:][::-1]
relevant_papers = df.iloc[top_n_indices]

# get least relevant papers
least_n_indices = cosine_similarities.argsort()[:100]
least_relevant_papers = df.iloc[least_n_indices]
# exclude papers with terms like "call for papers" or "conference"
# least_relevant_papers = least_relevant_papers[~least_relevant_papers['combined_text'].str.contains('teach|student|educat|pedagog|classroom|course|CS1', case=False)]


least_relevant_papers
relevant_papers
cosine_similarities


# sort by cosine similarity (descending)
df['cosine_similarity'] = cosine_similarities
df = df.sort_values(by='cosine_similarity', ascending=False)
df.head()

(4174, 5)


,source,title,url,json content,combined_text,cosine_similarity
1245,GitHub,Safe serialization,https://github.com/huggingface/optimum-quanto/...,"{""title"": ""Safe serialization"", ""url"": ""https:...","{""title"": ""Safe serialization"", ""url"": ""https:...",0.552801
2287,GitHub,[`core` / `Quantization`] Fix for 8bit seriali...,https://github.com/huggingface/transformers/pu...,"{""title"": ""[`core` / `Quantization`] Fix for 8...","{""title"": ""[`core` / `Quantization`] Fix for 8...",0.363702
2163,GitHub,Fix CPU offload + disk offload tests,https://github.com/huggingface/transformers/pu...,"{""title"": ""Fix CPU offload + disk offload test...","{""title"": ""Fix CPU offload + disk offload test...",0.304729
1488,GitHub,Add --safe-serialization/--no-safe-serialization,https://github.com/arcee-ai/mergekit/pull/103,"{""title"": ""Add --safe-serialization/--no-safe-...","{""title"": ""Add --safe-serialization/--no-safe-...",0.299877
3210,GitHub,Add serialization method to TensorShare,https://github.com/chainyo/tensorshare/pull/15,"{""title"": ""Add serialization method to TensorS...","{""title"": ""Add serialization method to TensorS...",0.290425
